In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import json
import pandas as pd
from pathlib import Path
from copy import deepcopy
from functools import partial

from bellek.utils import set_seed, jprint

set_seed(89)

In [3]:
pd.options.display.float_format = '{:,.3f}'.format

In [4]:
def load_result_dataframe(pattern: str = "ablation-prompting-technique-20240817-232745.jsonl") -> pd.DataFrame:
    dfs = [pd.read_json(filename, orient='records', lines=True) for filename in Path(".").glob(pattern)]

    # for run, df in enumerate(dfs):
    #     df["run"] = run + 1

    return pd.concat(dfs, ignore_index=True) 

In [9]:
raw_df = load_result_dataframe()
raw_df.rename(
    columns={
        'qdecomp': 'Q-decomp',
        'context': 'Context',
        'retrieval': 'Retrieval',
        'top_k': 'Top-k',
        'qa': 'Prompting',
        'run': 'Run',
        'exact_match': "EM",
        'f1': "F1",
    }, 
    inplace=True,
)
raw_df

,Context,Retrieval,Prompting,Run,EM,F1
0,paragraphs,groundtruth,standard,1,0.520,0.623
1,paragraphs,groundtruth,standard,2,0.490,0.601
2,paragraphs,groundtruth,standard,3,0.490,0.607
3,paragraphs,groundtruth,cot-zs,1,0.520,0.640
4,paragraphs,groundtruth,cot-zs,2,0.540,0.664
5,paragraphs,groundtruth,cot-zs,3,0.500,0.646
6,paragraphs,groundtruth,cot-fs,1,0.650,0.718
7,paragraphs,groundtruth,cot-fs,2,0.620,0.698
8,paragraphs,groundtruth,cot-fs,3,0.650,0.731
9,paragraphs,groundtruth,cte,1,0.630,0.746


In [11]:
df = raw_df.copy().drop(columns=['Context', 'Retrieval'])

In [12]:
with open("table-ablation-prompting-all.tex", 'w') as f:
    f.write(df.to_latex(index=False, float_format='%.3f'))

In [13]:
agg_df = df.drop(columns=['Run']).groupby(['Prompting']).mean()
agg_df

,EM,F1
Prompting,,
cot-fs,0.640,0.716
cot-zs,0.520,0.650
cte,0.630,0.744
standard,0.500,0.610


In [14]:
with open("table-ablation-prompting-agg.tex", 'w') as f:
    f.write(agg_df.reset_index().to_latex(index=False, float_format='%.3f'))